In [1]:
# pysimplegui build the Windows UI
import PySimpleGUI as sg      

# import serial comms package
import serial
import serial.tools.list_ports as list_ports

import time

In [2]:
def show_ports():
    '''
    show_ports: show available ports for serial communications
    
    Parameters:
    None
    
    Returns:
    None
    
    Side Effects:
    prints to standard output, one line per port
    '''
    ports = list(list_ports.comports() )
    _ = [print(p) for p in ports]
    return None
    
#end show_ports

In [3]:
def setup_comms_port():
    '''
    setup_serial_port: Setup to read data from Arduino serial line
    
    This functiions sets up a serial communications port for 
    PC to Arduino communication.
    
    Note this function must align with the corresponding Arduino sketch
    especially wrt BaudRate
    
    The COM3 port is used (appears to be default port for USB serial lines)
    
    Parameters:
    None
    
    Returns
    SerialPort object
    
    Side Effects
    Set up port COM3 at fast baudrate, clears
    any input buffer.  We do the latter, so that
    the data read from Arduino is roughly in step with
    Arduino writes
    '''
    serialPort = serial.Serial(port="com3", baudrate=38400,)
    serialPort.reset_input_buffer()
    
    return serialPort
    
#end setup_comms_port

In [ ]:
# screen update rate millisecs
SCREEN_READ_PERIOD = 50


#
# define GUI to show light levels
# bar is 100 (default font) chars wide, 200 pixels high

layout = [      [sg.Text('Light Meter', size=(12,1), font=("Helvetica", 100))],     
                [sg.ProgressBar(1000, orientation='h', size=(100,200), key='pbar',)],
                [sg.Text(str(23), size=(5,1), font=("Helvetica", 100), key='Light',)],      
                [sg.Cancel()]
         ]      

window = sg.Window('Light Meter', layout,)
progress_bar = window['pbar']

#
#  Set up Serial Port
show_ports()
#  serial_port = setup_comms_port()
with serial.Serial(port="com3", baudrate=38400,) as serial_port:
    serial_string = ''
    serial_port.reset_input_buffer()

    for i in range(1,1000000):
        # read values from serial port, and show in screen
        event, values = window.read(timeout=SCREEN_READ_PERIOD,)    

        # check for user exit
        if( event=='Cancel' or event is None):
            break
        #end if

        if(serial_port.in_waiting > 0):
            # print('data to read!')
            serial_string = serial_port.readline()
            serial_string = serial_string.decode('ascii')

            light = int(serial_string)
            # print(f' i={i}, light={light}')
            window['Light'].update(light )
            progress_bar.UpdateBar(light)

        else:
            # print(f' i={i}: No data to read ...')
            time.sleep(0.5)
        #end if

    #end for
#end with
# with clause should close serial port for us
window.close()


COM3 - USB Serial Device (COM3)


In [7]:
print(int('123'))

123


In [5]:
sg.ProgressBar?

Init signature:
sg.ProgressBar(
    max_value,
    orientation=None,
    size=(None, None),
    auto_size_text=None,
    bar_color=(None, None),
    style=None,
    border_width=None,
    relief=None,
    key=None,
    pad=None,
    visible=True,
    metadata=None,
)
Docstring:      Progress Bar Element - Displays a colored bar that is shaded as progress of some operation is made
Init docstring:
:param max_value: max value of progressbar
:type max_value: (int)
:param orientation: 'horizontal' or 'vertical'
:type orientation: (str)
:param size: Size of the bar.  If horizontal (chars wide, pixels high), vert (pixels wide, rows high)
:type size: Tuple[int, int]
:param auto_size_text: Not sure why this is here
:type auto_size_text: (bool)
:param bar_color: The 2 colors that make up a progress bar. One is the background, the other is the bar
:type bar_color: Tuple[str, str]
:param style: Progress bar style defined as one of these 'default', 'winnative', 'clam', 'alt', 'classic', 'vista', 'x